<h1>20156025_Sungho Park</h1>

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import random

file_data		= "mnist_train.csv"
handle_file	= open(file_data, "r")
data        		= handle_file.readlines()
handle_file.close()

size_row	= 28    # height of the image
size_col  	= 28    # width of the image

num_image	= len(data)
count       	= 0     # count for the number of images

#
# normalize the values of the input data to be [0, 1]
#
def normalize(data):

    data_normalized = (data - min(data)) / (max(data) - min(data))

    return(data_normalized)

#
# example of distance function between two vectors x and y
#
def distance(x, y):

    d = (x - y) ** 2
    s = np.sum(d)
    # r = np.sqrt(s)

    return(s)

#
# make a matrix each column of which represents an images in a vector form 
#
list_image  = np.empty((size_row * size_col, num_image), dtype=float)
list_label  = np.empty(num_image, dtype=int)

for line in data:

    line_data   = line.split(',')
    label       = line_data[0]
    im_vector   = np.asfarray(line_data[1:])
    im_vector   = normalize(im_vector)

    list_label[count]       = label
    list_image[:, count]    = im_vector    

    count += 1

# 
# plot first 100 images out of 10,000 with their labels
# 
f1 = plt.figure(1)

# for i in range(2):

#     label       = list_label[i]
#     im_vector   = list_image[:, i]
#     im_matrix   = im_vector.reshape((size_row, size_col))

#     plt.subplot(2,1, i+1)
#     plt.title(label)
#     plt.imshow(im_matrix, cmap='Greys', interpolation='None')

#     frame   = plt.gca()
#     frame.axes.get_xaxis().set_visible(False)
#     frame.axes.get_yaxis().set_visible(False)

# plt.show()

In [2]:
# 테스트 데이터 불러오기
file_data_test		= "mnist_test.csv"

handle_file	= open(file_data_test, "r")
data_test      		= handle_file.readlines()
handle_file.close()

size_row	= 28    # height of the image
size_col  	= 28    # width of the image

num_image_test = len(data_test)
count       	= 0     # count for the number of images

list_image_test = np.empty((size_row * size_col, num_image_test), dtype=float)
list_label_test  = np.empty(num_image_test, dtype=int)
    
for line in data_test:
    line_data_test = line.split(',')
    label_test     = line_data_test[0]
    im_vector_test = np.asfarray(line_data_test[1:])
    im_vector_test = normalize(im_vector_test)
    
    list_label_test[count]   = label_test
    list_image_test[:,count]      = im_vector_test
    count += 1

In [10]:
# train data 적용

num_zero = 0
A = np.zeros((num_image, size_row * size_col + 1), dtype = 'float')    # 60000 , 785 
b = np.ones((num_image,1),dtype = 'float')                             # 60000 , 1
x = np.zeros((size_row * size_col + 1,1))                             # 785 , 1

#A,b 만들기
for i in range(num_image):
    if list_label[i] != 0:
        b[i] = -1.0                                 # 라벨이 0이아니라면 -1
    A[i,0] = 1                                      # 상수 추가
    A[i,1:len(A[0,:])] = list_image[:,i]            # train data 추가
    
    for j in range(size_row * size_col):
        if list_image[j,i] == 0:
            list_image[j,i] = random.randrange(1,3)/255.0
    
#print(A[0])

In [11]:
#least Square
x = inv(np.matrix(A.T) * np.matrix(A)) * np.matrix(A.T) * np.matrix(b)

<h2> Using Training Data <br><br>
(1) Positive True<br>
(2) Positive False<br>
(3) Negative True<br>
(4) Negative False<br>
</h2>

In [21]:
dd = A*x

num_zero = 0
num_not_zero = 0
for i in range(num_image):
    if list_label[i] == 0:
        num_zero += 1            #0인거갯수
    else:
        num_not_zero += 1        


pt_count = 0
pf_count = 0
nt_count = 0
nf_count = 0

for i in range(num_image):
    if list_label[i] == 0:        # 실제 label이 0인것중에
        if dd[i] >= 0:             # 내가 0이라고 대답한 갯수
            pt_count += 1
        else:                          # 내가 0이 아니라고 대답한 갯수
            nf_count += 1
            
    elif list_label[i] != 0:       # 실제 label이 0이 아닌것중에
        if dd[i] >= 0:              # 내가 0이라고 대답한 갯수
            pf_count += 1
        else:                           # 내가 0이 아니라고 대답한 갯수
            nt_count += 1

print("Positive True  : ",pt_count,"/",num_zero,"=", pt_count / num_zero ,"=",int(pt_count / num_zero * 100),"%")
print("Positive False : ",pf_count,"/",num_not_zero,"=", pf_count / num_not_zero,"=",int(pf_count / num_not_zero* 100),"%")
print("Negative True  : ",nt_count,"/",num_not_zero,"=", nt_count / num_not_zero,"=",int(nt_count / num_not_zero* 100),"%")
print("Negative False : ",nf_count,"/",num_zero,"=", nf_count / num_zero,"=",int(nf_count / num_zero* 100),"%")

Positive True  :  5172 / 5923 = 0.8732061455343576 = 87 %
Positive False :  177 / 54077 = 0.003273110564565342 = 0 %
Negative True  :  53900 / 54077 = 0.9967268894354346 = 99 %
Negative False :  751 / 5923 = 0.1267938544656424 = 12 %


<h2> Using Testing Data <br><br>
(1) Positive True<br>
(2) Positive False<br>
(3) Negative True<br>
(4) Negative False<br>
</h2>

In [17]:
test1 = np.zeros((num_image_test,size_row * size_col + 1),dtype = 'float')
result = np.zeros(num_image_test, dtype = 'float')

num_zero = 0
num_not_zero = 0

for i in range(num_image_test):
    test1[i] = 1
    test1[i,1:len(test1[0,:])] = list_image_test[:,i]
    if list_label_test[i] == 0:
        num_zero += 1            #0인거갯수
    else:
        num_not_zero += 1        

result = test1 * x

pt_count = 0
pf_count = 0
nt_count = 0
nf_count = 0

for i in range(num_image_test):
    if list_label_test[i] == 0:        # 실제 label이 0인것중에
        if result[i] >= 0:             # 내가 0이라고 대답한 갯수
            pt_count += 1
        else:                          # 내가 0이 아니라고 대답한 갯수
            nf_count += 1
            
    elif list_label_test[i] != 0:       # 실제 label이 0이 아닌것중에
        if result[i] >= 0:              # 내가 0이라고 대답한 갯수
            pf_count += 1
        else:                           # 내가 0이 아니라고 대답한 갯수
            nt_count += 1

print("Positive True  : ",pt_count,"/",num_zero,"=", pt_count / num_zero ,"=",int(pt_count / num_zero * 100),"%")
print("Positive False : ",pf_count,"/",num_not_zero,"=", pf_count / num_not_zero,"=",int(pf_count / num_not_zero* 100),"%")
print("Negative True  : ",nt_count,"/",num_not_zero,"=", nt_count / num_not_zero,"=",int(nt_count / num_not_zero* 100),"%")
print("Negative False : ",nf_count,"/",num_zero,"=", nf_count / num_zero,"=",int(nf_count / num_zero* 100),"%")

Positive True  :  867 / 980 = 0.8846938775510204 = 88 %
Positive False :  42 / 9020 = 0.004656319290465632 = 0 %
Negative True  :  8978 / 9020 = 0.9953436807095344 = 99 %
Negative False :  113 / 980 = 0.11530612244897959 = 11 %
